In [ ]:
# -*- coding: utf-8 -*-
"""22_MovieRecommendationSystemUsingSVD.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/14f6XGC50pM0Uuu_dyy5F-OAGTdmHnB64

# 22 Movie Recommendation System Using SVD

### Importing the basic libraries
"""

import numpy as np
import pandas as pd

"""### Importing & Parsing the dataset as ratings and movies details"""

try:
    ratingData = pd.read_csv('ratings.dat', 
        names=['user_id', 'movie_id', 'rating', 'time'],
        engine='python', delimiter='::', encoding='ISO-8859-1')
    
    movieData = pd.read_csv('movies.dat',
        names=['movie_id', 'title', 'genre'],
        engine='python', delimiter='::', encoding='ISO-8859-1')

    print(ratingData)
except Exception as e:
    print(f"Error loading data: {e}")

"""### Create the ratings matrix of shape (m×u)"""

ratingMatrix = np.ndarray(
    shape=(np.max(ratingData.movie_id.values), np.max(ratingData.user_id.values)),
    dtype=np.uint8)
ratingMatrix[ratingData.movie_id.values - 1, ratingData.user_id.values - 1] = ratingData.rating.values
print(ratingMatrix)

"""### Subtract Mean off - Normalization"""

normalizedMatrix = ratingMatrix - np.asarray([(np.mean(ratingMatrix, 1))]).T
print(normalizedMatrix)

"""### Computing SVD"""

A = normalizedMatrix.T / np.sqrt(ratingMatrix.shape[0] - 1)
U, S, V = np.linalg.svd(A)

"""### Calculate cosine similarity, sort by most similar and return the top N"""

def similar(ratingData, movie_id, top_n):
    index = movie_id - 1  # Movie id starts from 1
    movie_row = ratingData[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', ratingData, ratingData))  # Einstein summation
    similarity = np.dot(movie_row, ratingData.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)  # Perform an indirect sort along the given axis (Last axis)
    return sort_indexes[:top_n]

"""### Select k principal components to represent the movies, a movie_id to find recommendations and print the top_n results"""

k = 50
movie_id = 2
top_n = 5

sliced = V.T[:, :k]  # Representative data
indexes = similar(sliced, movie_id, top_n)

print('Recommendations for Movie {0}: \n'.format(
    movieData[movieData.movie_id == movie_id].title.values[0]))
for id in indexes + 1:
    print(movieData[movieData.movie_id == id].title.values[0])


         user_id  movie_id  rating       time
0              1      1193       5  978300760
1              1       661       3  978302109
2              1       914       3  978301968
3              1      3408       4  978300275
4              1      2355       5  978824291
...          ...       ...     ...        ...
1000204     6040      1091       1  956716541
1000205     6040      1094       5  956704887
1000206     6040       562       5  956704746
1000207     6040      1096       4  956715648
1000208     6040      1097       4  956715569

[1000209 rows x 4 columns]
[[5 0 0 ... 0 0 3]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[ 3.57400662 -1.42599338 -1.42599338 ... -1.42599338 -1.42599338
   1.57400662]
 [-0.37152318 -0.37152318 -0.37152318 ... -0.37152318 -0.37152318
  -0.37152318]
 [-0.23874172 -0.23874172 -0.23874172 ... -0.23874172 -0.23874172
  -0.23874172]
 ...
 [-0.03278146 -0.03278146 -0.03278146 ... -0.0327814

In [2]:
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load MNIST dataset for handwritten digit recognition
mnist = fetch_openml('mnist_784')
X, y = mnist.data, mnist.target

# Normalize and split the dataset
X /= 255.0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Initialize variables for drawing
drawing = False
last_x, last_y = None, None
canvas = np.zeros((400, 400, 3), dtype=np.uint8)

def draw(event, x, y, flags, param):
    global drawing, last_x, last_y
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        last_x, last_y = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.line(canvas, (last_x, last_y), (x, y), (255, 255, 255), 15)
            last_x, last_y = x, y
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.line(canvas, (last_x, last_y), (x, y), (255, 255, 255), 15)

# Set up OpenCV window
cv2.namedWindow('Drawing')
cv2.setMouseCallback('Drawing', draw)

while True:
    # Create a blank image
    img = canvas.copy()
    # Show the canvas
    cv2.imshow('Drawing', img)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'):  # Clear the canvas
        canvas = np.zeros((400, 400, 3), dtype=np.uint8)
    elif key == ord('p'):  # Predict the drawing
        # Preprocess the image for prediction
        gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray, (28, 28))  # Resize to 28x28 pixels
        gray = gray.flatten() / 255.0  # Flatten and normalize
        prediction = knn.predict([gray])  # Predict using the classifier
        print(f'Predicted digit: {prediction[0]}')
    elif key == ord('q'):  # Exit
        break

cv2.destroyAllWindows()


c:\Users\Asus Laptop\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\datasets\_openml.py:72: RuntimeWarning: Invalid cache, redownloading file
  warn("Invalid cache, redownloading file", RuntimeWarning)
c:\Users\Asus Laptop\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\datasets\_openml.py:107: UserWarning: A network error occurred while downloading https://api.openml.org/data/v1/download/52667. Retrying...
  warn(


KeyboardInterrupt: 